In [1]:
##EXTRACTING SCRIPT -> IMAGECOLLECTION -> IMAGE -> GOOGLEDRIVE

In [10]:
import ee
import geemap
import os

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  # Get the pixel QA band.
  qa = image.select('pixel_qa')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

In [5]:
#MASKING FUNCTION FOR L5 AND L7
def cloudMaskL457(image):
  qa = image.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
  cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
  # Remove edge pixels that don't occur in all bands
  mask2 = image.mask().reduce(ee.Reducer.min())
  return image.updateMask(cloud.Not()).updateMask(mask2)

In [64]:
#setting variables
pointA = [12.37558, 45.51725]
pointB = [12.46914, 45.47393]

region = ee.Geometry.Rectangle([pointA[0], pointA[1], pointB[0], pointB[1]])
folder = "Test_Folder"
scale = 30

################ CHANGE THIS FOR HOW MANY ITERATION WILL BE MADE #####################
starting_year = 2019

In [87]:
def generateLandsatImage(year):
    print("entered")
    if year<1999:
        print("<1999")
        dataset = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image = dataset.select(['B3', 'B2', 'B1']).median()
        vis = {
          'bands': ['B3', 'B2', 'B1'],
          'gamma': 1.4,
          'max': 3000,
          'min': 0
        }
    elif year<2013 :
        print("<2013")
        dataset = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image = dataset.select(['B3', 'B2', 'B1']).median()
        vis = {
          'bands': ['B3', 'B2', 'B1'],
          'gamma': 1.4,
          'max': 3000,
          'min': 0
        }
    else :
        print("else")
        dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(maskL8sr) \

        image = dataset.select(['B4', 'B3', 'B2']).median()

        vis = {
          'bands': ["B4","B3","B2"],
          'gamma': 1.4,
          'max': 3000,
          'min': 0,
          'dimensions': 512,
          'region': region,
          'format': 'png'
        }
    print("return")
    url = image.getThumbURL(vis)
    print("return2")
    return image.visualize(vis)

In [88]:
for year in range(starting_year,2020,1):
    #gets right landset and exports image
      outImg = generateLandsatImage(year)
      url = outImg.getThumbURL({'scale': 10, 'region': region, 'format':'png'})
      print(url)

      #creates image of water map and exports
      dataset_water = ee.ImageCollection("JRC/GSW1_2/YearlyHistory") \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31')

      image_water = dataset_water.select(['waterClass']).median()

entered
else
return


EEException: Computation timed out.